In [1]:
# !conda create -n newCondaEnvironment -c cctbx202208 -y
# !source /opt/conda/bin/activate newCondaEnvironment && conda install -c cctbx202208 python=3.8.18 -y

# !sudo rm /opt/conda/bin/python3
# !sudo ln -sf /opt/conda/envs/newCondaEnvironment/bin/python3 /opt/conda/bin/python3

# !sudo rm /opt/conda/bin/python3.10
# !sudo ln -sf /opt/conda/envs/newCondaEnvironment/bin/python3 /opt/conda/bin/python3.10

# !sudo rm /opt/conda/bin/python
# !sudo ln -s /opt/conda/envs/newCondaEnvironment/bin/python3 /opt/conda/bin/python

# !python --version

In [2]:
!pip install openmim
!mim install mmdet
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -q -e .
!pip install clearml
!pip install -U torch==2.1
!pip install mmcv==2.1.0 -f https://download.openmmlab.com/mmcv/dist/cu121/torch2.1/index.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.1/443.1 kB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.1/953.1 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 6.3 MB/s eta 0:00:00
  Created wheel for oss2: filename=oss2-2.17.0-py3-none-any.whl size=112371 sha256=134f898b876a4a0070a7f13120b42d

In [3]:
!python3 --version

Python 3.10.13


In [4]:
!pip show  mmcv | grep Version

Version: 2.1.0


In [5]:
%%writefile configs/faster_rcnn/faster-rcnn_r50_fpn_drons_vs_berds.py

# The new config inherits the base configs to highlight the necessary modification
_base_ = [
    '../_base_/models/faster-rcnn_r50_fpn.py',
    # '../faster_rcnn/faster-rcnn_r18_fpn_8xb8-amp-lsj-200e_coco.py',
     '../_base_/default_runtime.py'
     
]
model = dict(
    type='FasterRCNN',
 
    roi_head=dict(bbox_head=dict( num_classes=1  ) ))

metainfo={
        'classes':
        (
    "drone",
        ),
        # palette is a list of color tuples, which is used for visualization.
        'palette':
        [(220, 20, 60)]
    }

albu_train_transforms = [
    dict(
        type='ShiftScaleRotate',
        shift_limit=0.0625,
        scale_limit=0.0,
        rotate_limit=25,
        interpolation=1,
        p=0.5),
    dict(
        type='Affine',
        translate_percent=0.2,
        rotate=(-25,25),
        shear=(-25,25),
        p=0.5),
    dict(
        type='InvertImg',
        p=0.2),
    dict(
        type='Equalize',
        p=0.2),
    dict(
        type='Solarize',
        p=0.2),
    dict(
        type='RandomBrightnessContrast',
        brightness_limit=[0.1, 0.3],
        contrast_limit=[0.1, 0.3],
        p=0.2),
    dict(
        type='OneOf',
        transforms=[
            dict(type='Blur', blur_limit=3, p=1.0),
            dict(type='MedianBlur', blur_limit=3, p=1.0)
        ],
        p=0.1),
    
]
#https://github.com/open-mmlab/mmdetection/issues/10340

 
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=False),
    # dict(
    #     type='Mosaic',
    #     img_scale=(1333, 1333) ),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
            type='BboxParams',
            format='pascal_voc',
            label_fields=['gt_bboxes_labels', 'gt_ignore_flags'],
            min_visibility=0.0,
            filter_lost_elements=True),
        keymap={
            'img': 'image',
            'gt_masks': 'masks',
            'gt_bboxes': 'bboxes'
        },
        skip_img_without_anno=True),
 
    dict(
        type='Resize',
        scale=1333,
        keep_ratio=True),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PackDetInputs'),

]
# train_dataset = dict(
#     # use MultiImageMixDataset wrapper to support mosaic and mixup
#     type='MultiImageMixDataset',
#     dataset=dict(
#         type='CocoDataset',
#         # data_root=data_root,
#         metainfo=metainfo,
#         ann_file='/home/vlad/datasets/drons_vs_berds/coco/dataset/drons_vs_berds_train.json',
#         data_prefix=dict(img='/home/vlad/datasets/drons_vs_berds/images/'),
#         pipeline=[
#             dict(type='LoadImageFromFile'),
#             dict(type='LoadAnnotations', with_bbox=True, with_mask=False),
#         ],
#         filter_cfg=dict(filter_empty_gt=False)),
#     pipeline=train_pipeline)


# set batch_size per gpu, and set new training pipeline
train_dataloader = dict(
    batch_size=4,
    num_workers=3,
    sampler=dict(type='DefaultSampler', shuffle=True),
    # over-write `pipeline` with new training pipeline setting
    dataset=dict(pipeline=train_pipeline,
                 type='CocoDataset',
                 metainfo=metainfo,
                 ann_file='/kaggle/input/drons-vs-berds-coco-ann/drons_vs_berds_train.json',
                 data_prefix=dict(img='/kaggle/input/images-drons-vs-berds/images'))
    )

# train_dataloader = dict(
#     batch_size=1,
#     num_workers=3,
#     sampler=dict(type='DefaultSampler', shuffle=True),
#     # over-write `pipeline` with new training pipeline setting
#     dataset=train_dataset)

val_dataloader = dict(
    batch_size=4,
    num_workers=3,
       
    # over-write `pipeline` with new training pipeline setting
    dataset=dict(pipeline=train_pipeline,
                 type='CocoDataset',
                 metainfo=metainfo,
                 test_mode=True,
                 ann_file='/kaggle/input/drons-vs-berds-coco-ann/drons_vs_berds_valid.json',
                 data_prefix=dict(img='/kaggle/input/images-drons-vs-berds/images')))
val_cfg = dict(type='ValLoop')
val_evaluator = dict(
    type='CocoMetric',
    ann_file='/kaggle/input/drons-vs-berds-coco-ann/drons_vs_berds_valid.json',
    metric=['bbox'],
   
    # format_only=False,
    # backend_args=None
    )
test_dataloader = val_dataloader
test_evaluator = val_evaluator
test_cfg = val_cfg
# Set optimizer
# optim_wrapper = dict(
#     type='OptimWrapper',
#     optimizer=dict(type='SGD', lr=0.01, momentum=0.9, weight_decay=0.0001))

optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(type='Adam', lr=0.001, weight_decay=0.0001))

#https://mmengine.readthedocs.io/zh-cn/v0.6.0/api/generated/mmengine.optim.OneCycleLR.html
param_scheduler = [
    dict(type='OneCycleLR',
         eta_max=0.01,
         total_steps=30000,
         by_epoch=False,
         pct_start=0.3,
         anneal_strategy='cos',
         div_factor=25,
         final_div_factor=1e4,
        #  three_phase=True
         )
]

vis_backends = [
    dict(type='LocalVisBackend'),
    dict(type='TensorboardVisBackend'),
    dict(type='ClearMLVisBackend',
    #      init_kwargs={
    #         'project': 'mmdetection',
    #         'group': 'maskrcnn-r50-fpn-1x-coco'
        #  }
         )
]
visualizer = dict(
    type='DetLocalVisualizer',
    vis_backends=vis_backends,
    name='visualizer')
 

# train, val, test loop config
train_cfg = dict(
    type='EpochBasedTrainLoop',  # The training loop type. Refer to https://github.com/open-mmlab/mmengine/blob/main/mmengine/runner/loops.py
    max_epochs=3,  # Maximum training epochs
    val_interval=1) 
log_level = 'INFO'
# We can use the COCO pre-trained Cascade Mask R-CNN R50 model for a more stable performance initialization
# load_from = 'https://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_mask_rcnn_r50_fpn_1x_coco/cascade_mask_rcnn_r50_fpn_1x_coco_20200203-9d4dcb24.pth'

Writing configs/faster_rcnn/faster-rcnn_r50_fpn_drons_vs_berds.py


In [6]:
!python   mmdet/utils/collect_env.py

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c107WarningC1ENS_7variantIJNS0_11UserWarningENS0_18DeprecationWarningEEEERKNS_14SourceLocationENSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEEb'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/opt/conda/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.1.0) or chardet (None)/charset_normalizer (3.3.2) doesn't match a supported version!
  warnings.warn(
sys.platform: linux
Python: 3.10.13 | packaged by conda-forge | (main, Dec 23 2023, 15:36:39) [GCC 12.3.0]
CUDA available: True
MUSA available: False
numpy_random_seed: 2147483648


In [7]:
from kaggle_secrets import UserSecretsClient
 
CLEARML_API_ACCESS_KEY = UserSecretsClient().get_secret('CLEARML_API_ACCESS_KEY')
CLEARML_API_SECRET_KEY = UserSecretsClient().get_secret('CLEARML_API_SECRET_KEY')

import os
os.environ['CLEARML_WEB_HOST'] = 'https://app.clear.ml'
os.environ['CLEARML_API_HOST'] = 'https://api.clear.ml'
os.environ['CLEARML_FILES_HOST'] = 'https://files.clear.ml'
os.environ['CLEARML_API_ACCESS_KEY'] = CLEARML_API_ACCESS_KEY
os.environ['CLEARML_API_SECRET_KEY'] = CLEARML_API_SECRET_KEY

In [8]:
!python tools/train.py configs/faster_rcnn/faster-rcnn_r50_fpn_drons_vs_berds.py

/opt/conda/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.1.0) or chardet (None)/charset_normalizer (3.3.2) doesn't match a supported version!
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c107WarningC1ENS_7variantIJNS0_11UserWarningENS0_18DeprecationWarningEEEERKNS_14SourceLocationENSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEEb'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
04/02 15:01:02 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 | packaged by conda